<a href="https://colab.research.google.com/github/YuehMintTai/000/blob/main/MongoDBTest20220201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok

In [ ]:
###https://dashboard.ngrok.com/get-started/setup!mkdir -p /drive/ngrok-ssh
##https://colab.research.google.com/github/shawwn/colab-tricks/blob/master/ngrok-tricks.ipynb#scrollTo=1B_aC562JbJU
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok
!./ngrok authtoken 24VntdRE0Mqj6buTgjDDBLZLSBN_4tX4P8h72ZYpEudaobfKd

[Errno 2] No such file or directory: '/drive/ngrok-ssh'
/content
--2022-02-01 14:30:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  8.37MB/s    in 1.6s    

2022-02-01 14:30:44 (8.37 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
cp: cannot stat '/drive/ngrok-ssh/ngrok': No such file or directory
chmod: cannot access '/ngrok': No such file or directory
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,make_response
app=Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
  return '哈囉你好嗎?'

if __name__=='__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://649e-35-196-232-152.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Feb/2022 14:30:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2022 14:30:54] "GET /favicon.ico HTTP/1.1" 404 -
